# Image similarity

In order to calculate e.g. image similarity for image search, we can extract features from a convolutional network. It's common to use features from the layer before the final classifier, e.g. the features circled in the diagram below.

![img](./images/googlenet_diagram_features2.png)

Extracting these features is very simple using Caffe, as we show below.

In [1]:
import numpy as np

# Make sure that caffe is on the python path:
caffe_root = '/home/ubuntu/caffe/' # this file is expected to be in {caffe_root}/examples
model_root = '/home/ubuntu/caffe/models/bvlc_googlenet/'
import sys
sys.path.insert(0, caffe_root + 'python')
import caffe

/usr/lib/python2.7/dist-packages/pkg_resources.py:1031: UserWarning: /home/ubuntu/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).
  warnings.warn(msg, UserWarning)


In [2]:
caffe.set_mode_gpu()
net = caffe.Net(model_root + 'deploy.prototxt', 
                model_root + 'bvlc_googlenet.caffemodel',
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB (opencv)

In [19]:
# insert an image
im = caffe.io.load_image('./images/ant_small.jpg')
im = transformer.preprocess('data', im)
net.blobs['data'].data[...] = im
# do forward pass
net.forward()
# extract features from layers
features = net.blobs['pool5/7x7_s1'].data[0,:,0,0]
features
# can use with e.g. cosine distance or euclidean distance for information retrieval

array([ 0.00657718,  4.47157335,  0.        , ...,  2.10681844,
        0.        ,  0.26480123], dtype=float32)